<font color='blue'>Привет! Поздравляю с первым выполненным проектом. Проделана большая работа. Меня зовут Алексей Алексеев, я проверю твою работу и напишу свои комментарии. Они, в зависимости от результата проверки, будут в </font><font color='green'>зеленом (когда все отлично),</font> <font color='brown'>коричневом (когда можно лучше)</font><font color='blue'> или</font> <font color='red'>красном (когда нужно переделать, чтобы я принял работу).</font><font color='blue'> Ну и в синем будут просто советы и нейтральные комментарии. Пожалуйста, не удаляй эти комментарии, они понадобятся нам для дальнейшей работы над проектом. Кстати, сразу предлагаю общаться на "ты".  
Наличие критичного или трех и более некритичных замечаний является поводом для отправки проекта на доработку. Некритичные замечания могут касаться нового материала, не освещенного в тренажере, но пожалуйста, не воспринимай их как мое необоснованное желание указать на твои ошибки. Изучать новый материал по ходу работы над проектом - это часть процесса обучения, на мой взгляд, очень эффективная. Так что не стесняйся задавать вопросы, если что-то не понятно. Наша общая цель - получить новые знания и навыки.     
А чтобы я точно не пропустил твои апдейты, желательно их также подсветить другим цветом, например, <font color='purple'>фиолетовым</font>.

---   
<font color='blue'>Апдейт после проверки: хороший проект, но давай еще немного поработаем над сводными.   
    Более подробно см. блок в конце.

</font>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
solvency = pd.read_csv('/home/artyom/datasets/data.csv')

print(solvency.head(20))
print()
solvency.info()

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

### Вывод

days_employed -- значения в данном поле, по всей видимости, на каком-то из этапов были сохранены с ошибкой (обилие отрицательных значений, нереалистичные значения в наутральных числах). Без понимания природы ошибки, использование и преобразование данных из этого поля не позволит получить конечный набор достоверных данных.

education -- предстоит выявить все возможные уровни образования, упомянутые в выборке, и унифицировать формат представления данных; установить корреляцию значений со значениями education_id.

family_status -- предстоит выявить все возможные значения семейного статуса в данной выборке, установить корреляцияю с полями family_status_id и gender

income_type -- предстоить выявить все перечисленные значения, унифицировать их, и возможно использовать для последующей категоризации данных.

debt -- за неимением прочих отсылок, будем считать данное поле Булевым. т.е. 0 -- False, нет задолженности, 1 -- True, есть задолженность.

total_income -- для наглядности данных, приведем поле к целочисленному формату.

purpose -- назначение кредита. Выявим все возможные цели кредитования.

<font color='green'>Данные загружены и подробно изучены, использованы подходящие методы для первичного изучения данных. Отлично подмечены основные проблемы.</font>

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
solvency.isna().sum() #проверим сколько всего в таблице пропущенных значений, и в каких полях

#Десятая часть данных о трудовом стаже и доходе не заполнена. 
#Удаление такого объема данных может существенно повлиять на результаты исследования. 
#Следовательно, необходимо найти подходящую замену недостающим данным.

#За неимением данных о трудовом стаже, заполним пропуски по полю days_employed медианным значением days_employed
solvency['days_employed'] = solvency['days_employed'].fillna(solvency['days_employed'].median())

#Аналогично, за неимением данных о доходах этих клиентов, положим их доходы равными медианному значению дохода по полю total_income
solvency['total_income'] = solvency['total_income'].fillna(solvency['total_income'].median())

#Вновь проверим наличие пропусков

solvency.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

В стобцах days_emplyed и total_income обнаружено много пропущенных значений. Значения по данным полям могли не указать непосредственно клиенты, на основе данных которых сформирована выборка, поскольку вместе созначением трудового стажа отсутствуют также и сведения о доходе. 

Тем не менее, удаление такого количества строк может повлиять на результаты анализа, поэтому пропущенные значения заменил медианным значением каждого столбца, т.к. данные столбцы содержат в себе количественные переменные.

<font color='green'>Очень хорошо. Молодец, что заполняешь медианами, а не средним арифметическим. 

<font color='brown'>Однако можно пойти немного дальше и попробовать заполнить пропуски медианами по роду деятельности: пенсионеров - медианами пенсионеров, сотрудников - медианами сотрудников. Так заполнение пропусков будет еще достовернее.
    
<font color='purple'> Привет! Спасибо, буду иметь ввиду. Будет полезно при возможной категорзиации по источникам доходов

### Замена типа данных

In [4]:
#Значения по полям days_employed и total_income для удобства восприятия приведём к целочисленному типу int

solvency['days_employed'] = solvency['days_employed'].astype('int')

solvency['total_income'] = solvency['total_income'].astype('int')

solvency.info() #все численные поля теперь имеют тип integer, тип object остался для строк

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Все поля типа данных с плавющей точкой float были приведены к типу целочисленному типу данных int для удобства анализа.

<font color='green'>Согласен, дробные части дней стажа и копейки в уровнях дохода нас не интересуют. А замена на целочисленный тип не только улучшает визуальное восприятие, но и сэкономит память и время.

### Обработка дубликатов

In [5]:
#посчитаем количество дубликатов в датафрейме

solvency.duplicated().sum()

#удалим дубликаты с сохранением порядка индексации

solvency=solvency.drop_duplicates().reset_index(drop=True)

#унифицируем запись текстовых значений во всех столбцах типа object

solvency['education'] = solvency['education'].str.lower()

solvency['family_status'] = solvency['family_status'].str.lower()

solvency['income_type'] = solvency['income_type'].str.lower()

solvency['purpose'] = solvency['purpose'].str.lower()

solvency=solvency.drop_duplicates().reset_index(drop=True) #ДОБАВЛЕНО: удаление дубликатов после унификации регистра

#ещё раз проверим на дубликаты весь датафрейм

solvency.duplicated().sum() #дубликатов нет

0

 ~<font color=red>Хмм, кажется, все же есть ;) Похоже, после приведениея к одному регистру появились новые дубликаты. Нужно что-то с ними сделать.~
    
 <font color=purple> Здесь я думал, что блок возвращает "17", когда считывается первый вызов solvency.duplicated().sum(), потому что при втором вызове возвращается уже значение 0 (я думал оно приходит из второго вызова solvency.duplicated().sum()) 
     
 <font color=purple> Добавил удаление дубликатов после унификации регистра.

<b>UPD: <font color='green'>Отлично, с этим разобрались.

### Вывод

Была проведена проверка выгрузки на наличие в ней дубликатов. Дублирующие строки были удалены методом drop_duplicates(). Во всех полях типа данных object все значения были обновлены в нижнем регистре, после чего вновь была выполнена проверка на наличие дубликатов.

### Лемматизация

In [6]:
#испортируем класс Mystem библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem()
#испортируем класс SnowballStemmer библиотеки nltk

#проверим, какие вообще уникальные цели указывали пользователи

print(solvency['purpose'].unique())

#для лемматизации запросов клиентов,создадим функцию, которая будет принимать аргументом строку датафрейма, и проводить лемматизацию нужного поля ['purpose']
def lemmatizer(row):
    return "".join(m.lemmatize(row['purpose']))

solvency['lemmas'] = solvency.apply(lemmatizer, axis=1) #параметром axis=1 применим функцию лемматизации ко всем строкам

solvency.head(10)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье\n
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье\n
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование\n
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба\n
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья\n


<font color=blue>Можно написать непосредственно 
    
    solvency['lemmas'] = solvency.apply(m.lemmatize, axis=1)
    
Так выйдет короче.
    
<font color=purple> Спасибо, буду знать. В этом блоке я сейчас изменил одну вещь в функции lemmatizer -- в поле lemmas теперь пишется строка с леммами, а не массив.

<b>UPD: <font color='blue'>В целом, с массивом, на мой взгляд, даже удобнее работать, так что это тоже был неплохой вариант.

### Вывод

Потребности пользователей, перечисленные в поле purpose лемматизировали -- привели к словарной форме -- это поможет категоризировать потребности.

### Категоризация данных

In [17]:
#Проведем категоризацию по потребностям клиентов

def lemmas_debt_category(row):
    lemma = row['lemmas'] 
    if 'авто' in lemma:
        return 'автомобиль'
 
    if 'свад' in lemma:
        return 'свадьба'
 
    if 'недвиж' in lemma:
        return 'недвижимость'
    
    if 'жил' in lemma:
        return 'жилье'
 
    if 'образ' in lemma:
        return 'образование'
  

row_values = ['дополнительное образование'] 
row_columns = ['lemmas']
row = pd.Series(data=row_values, index=row_columns) 
lemmas_debt_category(row) 
 
#проведем категоризацию по наличию и количеству детей
def children_category(row):
    children = row['children']
    if row['children'] == 0:
        return 'Нет детей'
    
    if row['children'] == 1:
        return 'Один ребенок'
    
    if 2 <= row['children'] <= 3:
        return 'Несколько детей'
    
    if row['children'] > 3:
        return 'Многодетный'
    
#проведем категоризацию по семейному положению
#проверим какие есть опции статуса семейного положения

#family_status_dict = solvency[['family_status', 'family_status_id']]
#print(family_status_dict.drop_duplicates().head(10))

# 0 -- женат / замужем
# 1 -- гражданский брак
# 2 -- вдовец / вдова
# 3 -- в разводе
# 4 -- не женат / не замужем

def family_status_category(row):
    family_status = row['family_status_id']
    if family_status == 0:
        return 'Есть семья'
    
    if family_status == 1:
        return 'Планирует семью'
    
    if family_status == 2:
        return 'Вдовец/ вдова'
    
    if family_status == 3:
        return 'Разведен(а)'
    
    if family_status == 4:
        return 'Нет имеет супруга(и)'
    
#проведем категоризацию по уровню дохода
#изучим уровни дохода

min_income = solvency['total_income'].min()
min_income #20667

max_income = solvency['total_income'].max()
max_income #2265604

median_income = solvency['total_income'].median()
median_income #145017

mean_income = solvency['total_income'].mean()
mean_income #165225.6006339144

#Из доступных данных удалось установить значения минимального, медианного, среднего, и максимального дохода клиентов.
#Будем считать,что те, кто получают менее медианы имееют доход ниже среднего,
#те, кто имеют доход в диапазоне от медианы до среднего значения, имеют средний доход,
#те, кто имеют доход выше среднего значения, имеют доходы выше средних

def income_category(row):
    income = row['total_income']
    if min_income <= income <= median_income:
        return 'Доходы ниже средних'
    
    if median_income < income <= mean_income:
        return 'Средние доходы'
    
    if income > mean_income:
        return 'Доходы выше среднего'

solvency['lemmas_debt_category'] = solvency.apply(lemmas_debt_category, axis=1)
solvency['children_category'] = solvency.apply(children_category, axis=1)
solvency['family_status_category'] = solvency.apply(family_status_category, axis=1)
solvency['income_category'] = solvency.apply(income_category, axis=1)


solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,lemmas_debt_category,children_category,family_status_category,income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n,жилье,Один ребенок,Есть семья,Доходы выше среднего
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n,автомобиль,Один ребенок,Есть семья,Доходы ниже средних
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n,жилье,Нет детей,Есть семья,Средние доходы
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n,образование,Несколько детей,Есть семья,Доходы выше среднего
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n,свадьба,Нет детей,Планирует семью,Средние доходы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,операция с жилье\n,жилье,Один ребенок,Планирует семью,Доходы выше среднего
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,сделка с автомобиль\n,автомобиль,Нет детей,Есть семья,Средние доходы
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость\n,недвижимость,Один ребенок,Планирует семью,Доходы ниже средних
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,на покупка свой автомобиль\n,автомобиль,Несколько детей,Есть семья,Доходы выше среднего


<font color=green>Итак, в целом, хороший принцип разбиения на группы. Мне понравилось, что вы выделил многодетные семьи и оталкивался от статистических показателей при установлении границ дохода.

<font color='blue'>Когда не вполне очевидно, как разбить сет, бывает удобно воспользоваться квантилями (как медиана, только бьет данные не на две, а на произвольное количество частей). Метод `describe`, примененный к столбцу, поможет разбить группу на 4 равные по количеству клиентов части. Или можно использовать непосредственно `.quantile(q=[0.25,0.5,0.75])`. Как видишь, тут уже можно разгуляться и указать произвольные границы в процентах.

<s><font color=red>Теперь о более важном. Мы проделали излишнюю категоризацию в том смысле, что не нужно дополнительно разбивать на категории "должник/не должник", эти данные уже есть в таблице. Как их использовать в сводных подскажу ниже. А пока давай сделаем простое разбиение на бездетных/с детьми/многодетных ну и остальные категории соответсвенно.

<font color=purple> Попробовал избавиться от излишней категоризации и упростил соответствующие функции -- оставил категоризацию только по потребностям, количеству детей, семейному положению и по уровню доходов

<b>UPD: <font color='green'>Хорошо, именно то, что я хотел увидеть.

### Вывод

Доступную информацию о клиентах разбил на категории -- по потребностям, по семейному положению, по количеству детей, и по уровням доходов. Далее, опираясь на эти категории, можно будет сделать выводы о кредитны историях клиентов.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
#children_debt_pivot = solvency.pivot_table(index='children_debt_category', columns = 'gender', values = 'debt', aggfunc ='count')
#children_debt_pivot.sort_values(by = 'F', ascending = False)

children_pivot_table = solvency.pivot_table(index='children_category', columns = 'debt', values='children', aggfunc= 'count')
children_pivot_table.columns = ['no_debt', 'debt']
children_pivot_table

children_pivot_table['debt_per_cent'] = children_pivot_table['debt'] / (children_pivot_table['no_debt'] + children_pivot_table['debt']) * 100
children_pivot_table['debt_per_cent'] = children_pivot_table['debt_per_cent'].round(1)
children_pivot_table

,no_debt,debt,debt_per_cent
children_category,,,
Многодетный,114,12,9.5
Несколько детей,2161,221,9.3
Нет детей,13028,1063,7.5
Один ребенок,4364,444,9.2


<s><font color=red>По данной табличке совсем непонятно, как хорошо/плохо отдают кредиты клиенты в зависимости от количества детей.

<font color='blue'>Нам нужно посмотреть  _процент_ просрочивших кредит в каждой группе. Иными словами, какой процент из всех бездетных (однодетных, двухдетных) допустил просрочку. Как его получить? Просто разделить количество просрочивших бездетных на общее количество бездетных. Напомню, что pandas позволяет без проблем делить датафреймы друг на друга. Теперь о том, как эти количества получить. Сделать разделение по долгу мы можем прямо на этапе построения сводной:  

In [18]:
#код ревьюера
children_table = solvency.pivot_table(index='children', columns = 'debt', values='dob_years', aggfunc= 'count')
children_table.columns = ['no_debt','debt']
children_table

,no_debt,debt
children,,
-1,46.0,1.0
0,13028.0,1063.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN
20,68.0,8.0


<font color=blue>В этой таблице есть все, чтобы получить искомый процент. Например, для бездетных это количество должников бездетных (1063) разделить на общее количество бездетных (1063+13044)
    Только конечно, считать мы должны не числа, а элементы таблиц. (`children_table['no_debt'][0]` и т.д.)

<font color=purple> Исправил формирование сводной таблицы по образцу с учетом выделенных категорий

### Вывод

<font color=purple> На основании сводной таблицы можно выявить следующий паттерн -- чаще всего должниками по кредитам оказываются клиенты, у которых есть дети. С количеством детей вероятность возникновения задолженности увеличивается.

<b>UPD: <font color='green'>Отлично справился, теперь рассчеты и выводы абсолютно верны.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [20]:
family_status_pivot_table = solvency.pivot_table(index='family_status_category', columns = 'debt', values = 'family_status_id', aggfunc ='count')
family_status_pivot_table.columns = ['no_debt', 'debt']

family_status_pivot_table['debt_per_cent'] = family_status_pivot_table['debt'] / (family_status_pivot_table['no_debt'] + family_status_pivot_table['debt']) * 100
family_status_pivot_table['debt_per_cent'] = family_status_pivot_table['debt_per_cent'].round(1)
family_status_pivot_table

,no_debt,debt,debt_per_cent
family_status_category,,,
Вдовец/ вдова,896,63,6.6
Есть семья,11408,931,7.5
Нет имеет супруга(и),2536,274,9.8
Планирует семью,3763,388,9.3
Разведен(а),1110,85,7.1


### Вывод

<font color=purple>По результатам сводной таблицы для категории "семейный статус" видно, что чаще всего должниками становятся не женатые/не замужние клиенты, а также те, кто только планирует завести семью. 

Клиенты у которых есть или была семья с меньшей вероятностью просрочат выплаты по кредитам.

<b>UPD: <font color='green'>Отлично подмечено, действительно, здесь видно, что наиболее надежные заемщики, это те, кто находится или когда-либо был в законном браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
income_category_pivot_table = solvency.pivot_table(index='income_category', columns = 'debt', values = 'total_income', aggfunc ='count')
income_category_pivot_table.columns = ['no_debt', 'debt']

income_category_pivot_table['debt_per_cent'] = income_category_pivot_table['debt'] / (income_category_pivot_table['no_debt'] + income_category_pivot_table['debt']) * 100
income_category_pivot_table['debt_per_cent'] = income_category_pivot_table['debt_per_cent'].round(1)
income_category_pivot_table

,no_debt,debt,debt_per_cent
income_category,,,
Доходы выше среднего,7085,594,7.7
Доходы ниже средних,10805,974,8.3
Средние доходы,1823,173,8.7


### Вывод

<font color=purple>На основании выполненного разбиения по категориям доходов в предоставленной выборке видно, что те, чей совокупный доход мене 165.226 у.е. с большей вероятностью просрочат выплату по кредиту.

- Как разные цели кредита влияют на его возврат в срок?

In [22]:
lemmas_debt_category_pivot_table = solvency.pivot_table(index='lemmas_debt_category', columns = 'debt', values = 'total_income', aggfunc ='count')
lemmas_debt_category_pivot_table.columns = ['no_debt', 'debt']

lemmas_debt_category_pivot_table['debt_per_cent'] = lemmas_debt_category_pivot_table['debt'] / (lemmas_debt_category_pivot_table['no_debt'] + lemmas_debt_category_pivot_table['debt']) * 100
lemmas_debt_category_pivot_table['debt_per_cent'] = lemmas_debt_category_pivot_table['debt_per_cent'].round(1)
lemmas_debt_category_pivot_table

,no_debt,debt,debt_per_cent
lemmas_debt_category,,,
автомобиль,3903,403,9.4
жилье,4152,308,6.9
недвижимость,5877,474,7.5
образование,3643,370,9.2
свадьба,2138,186,8.0


### Вывод

<font color=purple>Из потребностей клиентов можно установить, что быстрее всего клиенты оплачивают кредиты на приобритетние жилья, это также справделиво и для кредитов на прочую недвижимость.

С наибольшей вероятностью просрочка возникнет по кредитам на автомобили и образование.

<s><font color=red>Этот шаг требует тщательной доработки, инструкции выше.

<font color='green'>Доработка выполнена очень хорошо. Верные цифры, лаконичный код, четкие и правильные выводы. Уверенное использование агрегации данных.

### Шаг 4. Общий вывод

На основе полученной выгрузки данных статистики по клиентам банка, был проведен анализ, который включал в себя следующие этапы:

1) Поиск и обработка пропусков. Были обнаружены пропуски в полях данных 'days_employed' и 'total_income' -- так как в данных полях хранятся колличественные значения, пропуски были заменены соответствующим медианным значением по обоим полям.

2) Для удобства анализа численных значений, типы данных с плавующией точкой были приведены к целочисленному типу int. В датафрейме таким образом остались только строки типа object и целочисленные значения.

3) Нашел в таблице дублирующие записи (точное совпадение по всем полям, от количества дней трудового стажа, до суммарного дохода). Такие дублирующие записи могли возникнуть при заполнении пропущенных значений в полях 'days_employed' и 'total_income', или из-за ошибок вставки данных в базу.

4) Выявил все уникальные категории потребностей пользователей, провел их лемматизацию для удобства последующей категоризации данных

5) Категоризовал информацию о пользователях по параметрам:

- а) категории потребностей клиентов
- б) по наличию детей
- в) по семейному положению клиентов
- г) по уровням дохода. Категории уровней дохода сформировал на основании минимального, максимального, медианного и среднего значения дохода клиентов, указанных в поле 'total_income'.

6) На основе проведенных категоризаций, удалось составить сводные таблицы для разных параметров.

<font color=purple> ОБНОВЛЕННЫЙ ВЫВОД 
    
    ИТОГО: Удалось установить, что:

- с наибольшей вероятностью задолженность по кредитам возникнет у клиентов, у которых есть дети. При этом, с количеством детей увеличивается шанс невыплаты кредита;
- клиенты, у которых нет семьи, или которые планируют семью, менее дисциплинированы при выплате кредитов чем те, у кого уже есть или была семья;
- Наиболее вероятно, что задолженность по кредиту возникнет у тех, чей совокупный доход меньше среднего по выборке (165.226 у.е.)
- Больше всего вероятность возникновения задолженностей по платежам на автомобили, образование и свадьбы.

<b>UPD: <font color=green>Все верно, хорошо, что в доработанном общем выводе ты даешь четкий ответ на центральный вопрос, то есть бизнес совет клиенту - кто для него хороший клиент, кто плохой. Именно этого он от нас обычно и ждет.

<font color=green>Хороший структурированный вывод.

<font color=blue>Однако также потребует дополнения после доработки.

<font color=green>К оформлению работы вопросов нет: все аккуратно, код и комментарии в подходящих ячейках. Код читабелен, комментарии грамотны и понятны.

---
<font color='blue'>Итак, отличная работа, особенно для первого проекта. Ты показываешь прекрасное владение изучаемым материалом: уверенно пользуешься pandas, умеешь подготавливать, очищать, обогощать данные. Также ты правильно разбиваешь их на категории, грамотно применяя метод `apply()`.  
    Однако нам обязательно необходимо разобраться со сводными и получить на их основе ответ на центральный вопрос. Жду доработанный проект, успехов!

<b>UPD: <font color='blue'>Хорошая доработка и отличный старт выполнения проектов, принимаю работу. Впереди еще много интересных исследований. Успехов в дальнейшем прохождении курса!

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [х]  есть пояснение, какие пропущенные значения обнаружены;
- [х]  описаны возможные причины появления пропусков в данных;
- [х]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [х]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [х]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [х]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [х]  описан процесс лемматизации;
- [х]  данные категоризированы;
- [х]  есть объяснение принципа категоризации данных;
- [х]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [х]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [х]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [х]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [х]  в каждом этапе есть выводы;
- [х]  есть общий вывод.